# 16.2 huggingface自然语言处理预训练模型
## 16.2.1 huggingface简介

- **huggingface**是一家专注于自然语言处理（NLP）和机器学习的公司，成立于2016年。
- 该公司以开发高质量的预训练模型和提供易于使用的工具库而闻名。
  - huggingface强大之处在于其广泛的模型支持和易于使用的接口。
  - 极大降低使用先进 NLP模型的门槛。
- 特别是其Python库： `transformers`，包括了很多预训练的 NLP 模型。
  - 如BERT、GPT-2、T5等。
- huggingface已经共享了超100,000个预训练模型，10,000个数据集。
  - 涵盖了 NLP、计算机视觉、语音、时间序列、生物学、强化学习等领域。
- huggingface的官方网站：http://www.huggingface.co
- 在该网站包括各种资源：
  - **Datasets**：数据集及其下载地址。
  - **Models**：各个预训练模型。
  - **course**：免费的nlp课程。
  - **docs**：文档。  

## 16.2.2 huggingface环境配置与简单应用

- **安装：**huggingface的 `transformers` 库可以通过 pip 轻松安装。

In [3]:
# 如需安装，去掉下述代码的注释符号#然后运行
# !pip install transformers

In [1]:
import transformers
# 截止2023年10月22日的最新版本
transformers.__version__

'4.33.2'

- **导入库：**

In [2]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import torch

- **选择模型：** 选择想使用的预训练模型。
  - 比如如下代码导入BERT 模型：
  - 下载地址点击[此链接。](https://huggingface.co/bert-large-uncased/tree/main)

In [3]:
# 在windows下可以通过编辑HF_HOME环境变量更改模型文件下载保存的本地目录
# linux环境下类似，此处做法是先下载到本地，然后从本地路径直接加载
model_dir = '../huggingface/bert-large-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModel.from_pretrained(model_dir)

- **分词：**使用 Tokenizer对文本分词。

In [4]:
inputs = tokenizer("Hello, world!", return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 1010, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

- **前向传播：** 然后可以把分词后的结果传给模型。

In [5]:
outputs = model(**inputs)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2305,  0.1082, -0.5714,  ..., -1.1076, -0.6620,  0.2725],
         [-0.5496,  0.2575, -0.5169,  ..., -0.6421, -0.2396,  0.3067],
         [-0.4049,  0.8780, -0.4370,  ..., -1.1055, -0.2523,  0.6833],
         [-0.7911, -0.2327, -0.7287,  ..., -0.5525, -0.0323,  0.3933],
         [-0.2234,  0.1671,  0.2098,  ..., -0.4743, -0.4543,  0.3528],
         [-0.7747,  0.4422, -0.6589,  ..., -0.5992, -0.0422,  0.3475]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9527, -0.8253,  1.0000,  ..., -0.9993,  0.9847, -0.9723]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

- **结果解析：** 输出结果通常是一个包含多个元素的元组，比如最后一层的隐藏状态、池化输出等。

In [6]:
last_hidden_states = outputs.last_hidden_state
last_hidden_states

tensor([[[-0.2305,  0.1082, -0.5714,  ..., -1.1076, -0.6620,  0.2725],
         [-0.5496,  0.2575, -0.5169,  ..., -0.6421, -0.2396,  0.3067],
         [-0.4049,  0.8780, -0.4370,  ..., -1.1055, -0.2523,  0.6833],
         [-0.7911, -0.2327, -0.7287,  ..., -0.5525, -0.0323,  0.3933],
         [-0.2234,  0.1671,  0.2098,  ..., -0.4743, -0.4543,  0.3528],
         [-0.7747,  0.4422, -0.6589,  ..., -0.5992, -0.0422,  0.3475]]],
       grad_fn=<NativeLayerNormBackward0>)

- **应用场景：** 使用这些输出在各种 NLP 任务上，如文本分类、命名实体识别（NER）、问答系统、情感分类等。

- **示例：BERT模型在情感分析中的应用**

In [7]:
# 加载预训练模型和分词器
model_dir = '../huggingface/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
text = "I love programming. It's amazing!"
# 分词和准备输入数据
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
# 使用模型进行预测
with torch.no_grad():
    outputs = model(**inputs)
# 解析模型输出，获取预测结果
logits = outputs.logits
predicted_class_idx = torch.argmax(logits, dim=1).item()

# "nlptown/bert-base-multilingual-uncased-sentiment" 模型有5个标签，从非常负面 (1 star) 到非常正面 (5 stars)
print(f"预测情感分析结果: {predicted_class_idx + 1} stars")


预测情感分析结果: 5 stars


## 16.2.3 huggingface生成式大语言模型
- **大语言模型（Large Language Models，简称LLM）**是深度学习中的自然语言处理模型，此种模型具有大量的参数，通常达到数十亿甚至数百亿，最新的GPT-4已达18000亿。这些模型通过在大量文本数据上进行训练，学会了生成和理解文本的能力。
- **类型：**
  - **Transformer 架构**：Transformer 是当前大多数大语言模型背后的关键架构，如 BERT、GPT 和 T5。
  - **基于 BERT 的模型**：如 RoBERTa、DistilBERT 和 ALBERT。它们是 BERT 的变种，有些进行了结构优化，有些进行了训练策略的优化。
  - **基于 GPT 的模型**：如 GPT-2、GPT-3、GPT-Neo和GPT-4等。
- **原理：**
  - **Transformer 架构**：Transformer 使用了自注意力机制来捕捉文本中的长距离依赖关系。
    - 它可以并行处理输入序列中的所有单词，而不是像传统的递归或卷积神经网络那样按顺序处理。
  - **预训练和微调**：大多数大语言模型的训练分为两个阶段：预训练和微调。
    - 在预训练阶段，模型在大量的无标记文本上进行训练，如网上的文章和书籍。
    - 在微调阶段，模型在特定任务的小型标记数据集上进行训练，如情感分类或问题回答。
- **应用：**
  - **文本生成**：如自动写作、诗歌创作和代码生成。
  - **问答系统**：可以提供特定的问题答案。
  - **机器翻译**：翻译不同的语言。
  - **情感分析**：确定文本的情感倾向，如正面、负面或中性。
  - **文本摘要**：自动生成文本的简短摘要。
  - **命名实体识别**：从文本中识别和分类实体，如人名、地点或日期。
  - **代码补全和错误修正**：在编程中辅助开发者。
- **生成式大语言模型与生成式AI**：
  - **生成式大语言模型**是指那些能够生成连贯文本或语言序列的模型。
    - 它们通常是训练过的，以根据给定的上下文（例如，前几个单词或句子）生成后续的文本或语言输出。
    - GPT系列（如GPT-2、GPT-3、基于GPT-4的ChatGPT）就是此类模型的典型例子。
  - **生成式AI**更广泛地指的是任何能够生成数据（如文本、图像、音频等）的人工智能系统。
    - 这些系统不仅限于文本生成，还包括音乐合成、艺术创作、图像生成等。
    - 例如，DCGANs（深度卷积生成对抗网络）是用于生成图像的生成式AI。
  - 生成式大语言模型通常基于Transformer架构，特别是其**解码器部分**。而生成式AI可能使用各种技术，如生成对抗网络（GANs）、变分自编码器（VAEs）等。
  - 生成式大语言模型主要关注文本生成，而生成式AI涵盖了所有类型的数据生成，不仅仅是文本。
  - 生成式大语言模型通常用于聊天机器人、文本补全、写作助手等，而生成式AI的应用范围**更广泛**，包括图像生成、音乐合成、视频编辑等。

### 16.2.3.1 ChatGLM3-6B-32K模型
- <b> ChatGLM3-6B-32K</b>是由是智谱AI和清华大学 KEG 实验室联合发布的新一代对话预训练模型。
- huggingface上该模型的主页标明研发组织为Knowledge Engineering Group (KEG) & Data Mining at Tsinghua University。
- 该模型在ChatGLM3-6B的基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32K长度的上下文。
- 该模型对位置编码进行了更新，并设计了更有针对性的长文本训练方法，在对话阶段使用 32K 的上下文长度训练。
- 在实际的使用中，如果面临的上下文长度基本在 8K 以内，推荐使用ChatGLM3-6B；如果需要处理超过 8K 的上下文长度，推荐使用ChatGLM3-6B-32K。
- ChatGLM3-6B 是 ChatGLM 系列最新一代的开源模型，在保留了前两代模型对话流畅、部署门槛低等众多优秀特性的基础上，ChatGLM3-6B 引入了如下特性：
  - 更强大的基础模型： ChatGLM3-6B 的基础模型 ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。在语义、 数学、推理、代码、知识等不同角度的数据集上测评显示，ChatGLM3-6B-Base 具有在 10B 以下的预训练模型中最强的性能。
  - 更完整的功能支持： ChatGLM3-6B 采用了全新设计的 Prompt 格式，除正常的多轮对话外。同时原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。
  - 更全面的开源序列： 除了对话模型 ChatGLM3-6B 外，还开源了基础模型 ChatGLM-6B-Base、长文本对话模型 ChatGLM3-6B-32K。

- 模型下载：
  - ChatGLM3-6B-32K模型在huggingface上的下载地址在[此处](https://huggingface.co/THUDM/chatglm3-6b-32k/tree/main)。
  - 或者按照该模型在modelscope上的[主页](https://modelscope.cn/models/ZhipuAI/chatglm3-6b-32k/summary)提供的方式下载，此种下载方式快很多，而且访问方便。

In [8]:
# 安装sentencepiece包
#!pip install sentencepiece

In [9]:
from transformers import AutoTokenizer, AutoModel
model_path = "../huggingface/chatglm3-6b-32k"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)

你好👋！我是 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。
晚上睡不着，可以尝试以下方法来放松身心，帮助入睡：

1. 建立规律的作息时间：每天按时上床睡觉和起床，有助于调整身体的生物钟。
2. 避免刺激性物质：睡前不要喝咖啡、茶或其他含咖啡因的饮料，也不要吃过多油腻的食物，这些都有助于放松身心。
3. 创造舒适的睡眠环境：保持卧室安静、温暖且适度暗淡，有助于入睡。
4. 放松身心：可以尝试深呼吸、冥想、瑜伽等方式来放松身心，减轻压力。
5. 避免过度使用电子设备：睡前不要过度使用手机、电脑等电子设备，这些设备发出的蓝光会抑制褪黑激素的分泌，影响睡眠质量。

如果长期存在睡眠问题，建议咨询医生或专业人士的意见。


In [11]:
response, history = model.chat(tokenizer, "鲁迅和周树人是什么关系？", history=history)
print(response)

鲁迅和周树人都是中国现代文学的奠基人，鲁迅是他的笔名，周树人则是他的本名。周树人原名周樟寿，字豫才，浙江绍兴人。他是中国文学史上著名的文学家、思想家、革命家，被誉为“二十世纪中国文学的奠基人”。鲁迅是他的笔名，他的代表作品包括小说《狂人日记》、《呐喊》、《彷徨》等，散文集《朝花夕拾》等。他的作品深刻地揭示了中国社会的黑暗面，对中国现代文学的发展产生了深远的影响。


In [12]:
response, history = model.chat(tokenizer, "张三的妈妈是李四的姐姐，张三和李四之间是什么关系？", history=history)
print(response)#一本正经地胡说八道

根据题意，张三的妈妈是李四的姐姐，那么张三和李四之间的关系可以用“堂兄妹”来表示。堂兄妹是指父亲的兄弟（或姐妹）和父亲的姐妹所生的子女，属于三代以内的旁系血亲。


### 16.2.3.2 Llama2 13B模型

- **Llama2**大语言模型由Meta开发并公开发布，是预训练和微调的生成文本模型集合。
- 此处微调后的LLM称为Llama-2-Chat，专为对话用例进行优化。
- 在测试的大多数基准测试中，Llama-2-Chat模型的表现超过了开源的聊天模型（英文领域）。
- 在人类评价中，其在有用性和安全性上与一些受欢迎的闭源模型如ChatGPT和PaLM相当。
- 模型架构： Llama 2是一个自回归语言模型，使用了优化的Transformer架构。
- 调优版本使用监督式微调（SFT）和带有人类反馈的强化学习（RLHF）来使模型与人类在有用性和安全性上的偏好对齐。
- 包含Llama2 7B, Llama2 13B和Llama2 70B三个版本，分别是70亿，130亿和700亿个参数。
- 此处使用13B进行测试，下载地址点击[此处](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf/tree/main)。

In [15]:
# 需要安装accelerate包
#!pip install accelerate

In [1]:
import transformers
import torch
from transformers import AutoTokenizer

model_directory = "../huggingface/Llama-2-13b-chat-hf"
llama_tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_directory,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=llama_tokenizer.eos_token_id,
    max_length=500,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I'm looking for something with a similar tone and style to "Breaking Bad" and "Band of Brothers". Here are some shows that you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss and his family. It has a similar tone and style to "Breaking Bad" and features complex characters and intricate storylines.
2. "The Wire" - This HBO series is a gritty and realistic portrayal of the drug trade in Baltimore and the people involved in it. It has a similar tone and style to "Breaking Bad" and features complex characters and intricate storylines.
3. "Narcos" - This Netflix series is a true-story drama that follows the rise and fall of Colombian drug lord Pablo Escobar and the DEA agents who hunted him down. It has a similar tone and style to "Breaking Bad" and features complex characters and intricate storylines.
4. "Pea

In [4]:
sequences = pipeline(
    '鲁迅和周树人是什么关系？',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=llama_tokenizer.eos_token_id,
    max_length=1000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 鲁迅和周树人是什么关系？  


























































































































































































子���������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������人��ED�人�����edor
ED













ED
edor。����������edor。����������������������������������������������������th
th�������������������������������th


































































































th


th
th









th

th

### 16.2.3.3 AquilaChat2-7B、34B模型
- 由北京智源人工智能研究院研究开发的大语言模型Aquila2。
- 此处使用为对话微调的版本，参数数量为7B。
- Aquila2还有一个34B的模型，但是在单片A800上无法加载模型，显存应该是够了，有可能是CPU内存不够。
- 以后有时间再研究，初步探索原因有可能是flagai在加载模型时一次性通过CPU内存进行转储，但是本机服务器内存只有64G，无法转储。
- 本例是 AquilaChat2-7B模型，效果差强人意。

In [1]:
from flagai.auto_model.auto_loader import AutoLoader
model_name = 'AquilaChat2-7B'

autoloader = AutoLoader("aquila2", model_name=model_name,model_dir= '../huggingface')
model = autoloader.get_model()
tokenizer = autoloader.get_tokenizer()

[2023-10-30 15:35:01,719] [INFO] [logger.py:85:log_dist] [Rank -1] Unsupported bmtrain


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
test_data = [
    "北京的十大景点是什么?请将回答翻译成英文和日语",
    "写一首中秋主题的五言绝句",
    "Write a tongue twister that's extremely difficult to pronounce.",
]

for text in test_data:
    print(model.predict(text, tokenizer=tokenizer, model_name=model_name))

model in: <|startofpiece|>北京的十大景点是什么?请将回答翻译成英文和日语<|endofpiece|>
北京的十大景点:

英语：
1. 故宫博物院
2. 颐和园
3. 天坛
4. 长城
5. 奥林匹克公园
6. 中国国家博物馆
7. 南锣鼓巷
8. 前门大街
9. 北京动物园
10. 中央电视塔

日语：
1. 北京故宫（紅寶門）
2. 北京颐和园（豊園門）
3. 北京天坛（祈年殿）
4. 北京长城（長城）
5. 北京奥林匹克公园（奧林匹克廣場）
6. 北京國家博賢院（國家博賢院）
7. 北京南锣鼓巷（南锣鼓巷）
8. 北京前门大街（前門大街）
9. 北京動物�
model in: <|startofpiece|>写一首中秋主题的五言绝句<|endofpiece|>
明月挂天边，皎洁洒银辉。
花香漫庭院，团圆在此夕。
model in: <|startofpiece|>Write a tongue twister that's extremely difficult to pronounce.<|endofpiece|>
ยังไม่รู้ว่ามันถูกส่งไปที่ทำงานหลังจาก เดิมคิอมีตัว ห้องหน้า ที่ห้องป๋อแน่นอน พร้อมกับหลอมแปลงร่างหนังสือที่สุดตั้งค่าแล้ว สบายใจยังไม่รู้ว่าจะส่งตัวไปยังทำงานหลัง ซึ่งอีกหนึ่งคนยั


In [3]:
model.predict("鲁迅和周树人是什么关系？", tokenizer=tokenizer, model_name=model_name)

model in: <|startofpiece|>鲁迅和周树人是什么关系？<|endofpiece|>


'鲁迅（1881年9月25日-1936年10月19日）和周树人（1897年11月12日-1972年2月16日）是中国现代文学史上著名的文学家，他们共同度过了自己的创作青春，并且在身后留下了深远的影响。鲁迅，原名周树人，字豫才，是中国现代文学的奠基人之一。他以深刻而犀利的笔触描绘了民国时期社会和人性的问题，批判现实，呼唤人性的救赎。周树人，字达生，是新文化运动时期的重要作家、散文家和教育家。他以细腻的笔触书写了许多现实主义小说和散文，探索了人性中的矛盾和冲突，同时也表达了对自然、历史、社会等方面的兴趣和理解。鲁迅与周树人的作品各具特色，但都充满了对人性的关注与思考。'

In [4]:
model.predict("张三的妈妈是李四的姐姐，张三和李四之间是什么关系？", tokenizer=tokenizer, model_name=model_name)

model in: <|startofpiece|>张三的妈妈是李四的姐姐，张三和李四之间是什么关系？<|endofpiece|>


'张三和李四之间是姐弟关系。因为张三的妈妈（李四的姐姐）是李四的姐姐，所以张三和李四之间是姐弟关系。'

### 16.2.3.4 Chinese-LLaMA-Alpaca-2模型
- 2023年07月31日，哈工大讯飞联合实验室，发布Chinese-LLaMA-Alpaca-2（简称Alpaca-2），本项目基于Meta发布的可商用大模型Llama-2开发，是中文LLaMA&Alpaca大模型的第二期项目，开源了中文LLaMA-2基座模型和Alpaca-2指令精调大模型。
  - 该模型在原版Llama-2的基础上扩充并优化了中文词表，使用了大规模中文数据进行增量预训练，进一步提升了中文基础语义和指令理解能力，相比一代相关模型获得了显著性能提升。
  - 相关模型支持FlashAttention-2训练。
  - 标准版模型支持4K上下文长度，长上下文版模型支持16K上下文长度，并可通过NTK方法最高扩展至24K+上下文长度。
- 详细资料可参阅[此文](https://yunyaniu.blog.csdn.net/article/details/132554196?spm=1001.2101.3001.6650.1&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-132554196-blog-132379631.235%5Ev38%5Epc_relevant_anti_t3&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-132554196-blog-132379631.235%5Ev38%5Epc_relevant_anti_t3&utm_relevant_index=2&ydreferer=aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3NoaWNoYW9nL2FydGljbGUvZGV0YWlscy8xMzIzNzk2MzE%2Fc3BtPTEwMDEuMjAxNC4zMDAxLjU1MDI%3D&ydreferer=aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3NoaWNoYW9nL2FydGljbGUvZGV0YWlscy8xMzIzNzk2MzE%2Fc3BtPTEwMDEuMjAxNC4zMDAxLjU1MDI%3D)。
- 该模型权重文件在huggingface上的[卡片主页](https://huggingface.co/hfl/chinese-alpaca-2-13b)。
- 该模型的相关源代码在[此处](https://github.com/ymcui/Chinese-LLaMA-Alpaca-2)，该仓库的[wiki页面](https://github.com/ymcui/Chinese-LLaMA-Alpaca-2/wiki)信息很丰富，值得一读。

- 本例是基于Chinese-Alpaca-2-13B (full model), 模型的[下载网页](https://huggingface.co/hfl/chinese-llama-2-13b/tree/main)。
<img src='../img/16_2_1.png' width=600px height=500px/>


- **示例1：在Linux的终端命令行运行Alpaca-2，未使用vLLM**
  - (1) 从github下载Alpaca-2模型的代码，从huggingface下载权重文件和配置文件。
  <img src='../img/16_2_2.png' width=600px/>
  - (2) 找到inference_hf.py文件，并记住名字，后面要用到。
  <img src='../img/16_2_3.png' width=600px/>
  - (3) 找到模型权重文件和配置文件的路径，并记住，后面要用到。
  <img src='../img/16_2_4.png' width=600px/>
  - (4) 运行模型，注意将base_model参数指向你本地机上模型权重文件夹的路径。<br>命令如下：<br>python3 inference_hf.py --base_model  /home/user/python/d2l/huggingface/chinese-alpaca-2-13b --with_prompt --interactive <br>  
  <img src='../img/16_2_5.png' width=600px/>
  - (5) 运行上述命令后，程序加载模型需要几分钟时间。
  <img src='../img/16_2_6.png' width=600px/>
  - (6) 进入问答对话界面，使用最简单的单轮问答功能测试该模型。
  <img src='../img/16_2_7.png' width=600px/>
  - (7) 使用nvidia-smi命令查看GPU参数，结果显示模型及其权重占用显存约28G。
  <img src='../img/16_2_8.png' width=600px/>
  - (8) 试用模型对话能力，推理能力差强人意。
  <img src='../img/16_2_9.png' width=600px/>
  <img src='../img/16_2_10.png' width=600px/>
  <img src='../img/16_2_11.png' width=600px/>
  <img src='../img/16_2_12.png' width=600px/>
  

- **示例2：使用vLLM运行Alpaca-2**
  - (1) 运行模型，在前例命令基础上增加use_vllm参数。vLLM可以参考[此文](https://www.zhihu.com/question/611905691?utm_id=0)，源代码仓库在[这里](https://github.com/vllm-project/vllm)。
  <br>命令如下：<br>python3 inference_hf.py --base_model  /home/user/python/d2l/huggingface/chinese-alpaca-2-13b --with_prompt --interactive --use_vllm<br>
  <img src='../img/16_2_13.png' width=600px/>
  - (2) 使用nvidia-smi命令查看GPU参数，显存占用显著增加，达73.8G
  <img src='../img/16_2_14.png' width=600px/>
  

  - (3) 试用模型对话能力，推理能力和准确率明显改善。
  <img src='../img/16_2_16.png' width=600px/>
  <img src='../img/16_2_17.png' width=600px/>

### 16.2.3.5 通义千问-14B-Chat-Int8(Qwen-14B-Chat-Int8)
- **通义千问-14B-Chat-Int8**，是**Qwen-14B模型的**的会话量化版，是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。
- 在Qwen-14B的基础上，使用对齐机制打造了基于大语言模型的AI助手Qwen-14B-Chat。
- 此处的模型Qwen-14B-Chat的Int8量化模型。
- 模型下载可以参考Qwen-14B-Chat-Int8在modelscope网站的[模型主页](https://modelscope.cn/models/qwen/Qwen-14B-Chat-Int8/summary)，下载速度很快。
- 运行模型的依赖包如下：
  - modelscope, transformers(4.32.0及以上版本), accelerate, tiktoken, einops, scipy, transformers_stream_generator==0.0.4, peft deepspeed, auto-gptq, optimum.
  - 其中deepspeed包要求事先配置好CUDA_HOME环境变量，但是安装失败似乎也不影响测试运行。DeepSpeed是一个深度学习优化库，它使分布式训练简单和高效，而无需进行大量代码修改。它被设计用于运行大规模的模型，尤其是那些超过单个GPU内存的模型。'

- 下载模型：
  - 使用modelscope包的snapshot_download函数下载模型，默认下载本地的/home/user/.cache/modelscope/hub目录下。
  - 注意将user替换成你的linux登录用户名。
  - 本例将模型拷贝到项目的huggingface目录下统一管理。
  - 下载代码如下：
```
from modelscope import snapshot_download
model_dir = snapshot_download("qwen/Qwen-14B-Chat-Int8", revision = "v1.0.0")
```

In [1]:
# pip install 'modelscope>=1.9.3'
from modelscope import AutoTokenizer, AutoModelForCausalLM
model_path="../huggingface/Qwen-14B-Chat-Int8"
# 从本地加载模型，包括权重文件和分词器
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, revision = 'master')
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    revision='master'
).eval()

2023-10-30 15:38:56,594 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-10-30 15:38:56,597 - modelscope - INFO - Loading ast index from /home/user/.cache/modelscope/ast_indexer
2023-10-30 15:38:56,766 - modelscope - INFO - Loading done! Current index file version is 1.9.4, with md5 450c3d8b1ea957822a87246807bcd95c and a total number of 945 components indexed
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
# 个人认为，在本视频测试的这几个模型中，通义千问回答此问题的效果最佳
response, history = model.chat(tokenizer, "鲁迅和周树人是什么关系？", history=None)
print(response)

鲁迅是周树人的笔名。周树人，字豫山，后改豫才，浙江绍兴人，著名文学家、思想家、革命家、教育家、民主战士，新文化运动的重要参与者，中国现代文学的奠基人之一。

1918年5月，首次以“鲁迅”作笔名，发表中国现代文学史上第一篇白话小说《狂人日记》。他的著作主要以小说、杂文为主，代表作有：小说集《呐喊》《彷徨》《故事新编》；散文集《朝花夕拾》；散文诗集《野草》；杂文集《坟》《热风》《华盖集》《华盖集续编》《南腔北调集》《三闲集》《二心集》《而已集》《且介亭杂文》等。


In [4]:
response, history = model.chat(tokenizer, "张三的妈妈是李四的姐姐，张三和李四之间是什么关系？", history=None)
print(response)

张三和李四是姐弟关系。


In [5]:
response, history = model.chat(tokenizer, "请推荐6部和权力的游戏类似的美剧，并进行简单介绍", history=None)
print(response)

1. 《纸牌屋》：该剧以华盛顿特区为背景，讲述了主角弗兰克·安德伍德如何通过各种手段，一步步走向权力巅峰的故事。

2. 《风骚律师》：这部剧是《绝命毒师》的衍生剧，讲述了一个曾被陷害入狱的小律师，重新站起来后如何挑战法律、挑战强权的故事。

3. 《西部世界》：这是一部科幻题材的电视剧，描绘了一个由机器人主宰的世界，人类在这个世界中失去了自由和尊严。

4. 《黑镜》：这部剧集通过一系列独立的故事，探讨了科技对现代社会的影响以及人性的弱点。

5. 《神盾局特工》：这是一部超级英雄题材的电视剧，讲述了神盾局如何对抗邪恶势力，保护世界和平的故事。

6. 《绿箭侠》：这是一部动作冒险题材的电视剧，讲述了一位富有的企业家成为正义的绿箭侠，对抗罪恶的故事。


### 16.2.3.6 百川2-13B-Chat(Baichuan2-13B-Chat)
- Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。
- Baichuan 2 在多个权威的中文、英文和多语言的通用、领域 benchmark 上取得同尺寸最佳的效果。
- 在开源网站modelscope和huggingface上发不了该模型的7B、13B 的 Base 和 Chat 版本，并提供了 Chat 版本的 4bits 量化。
- 2023年10月30日，百川智能总裁宣布推出Baichuan2-192K大模型，其上下文窗口长度高达192K，能够处理约35万个汉字。
- Baichuan2-192K是目前全球最长的上下文窗口，也是目前支持长上下文窗口最优秀大模型Claude2（支持100K上下文窗口，实测约8万字）的4.4倍，更是GPT-4（支持32K上下文窗口，实测约 2.5万字）的14倍（1400%）。
- [百川自称](https://www.toutiao.com/article/7295588931813622312/?app=news_article&timestamp=1698641617&use_new_style=1&req_id=202310301253365697EC7A0D5F0956898F&group_id=7295588931813622312&wxshare_count=1&tt_from=weixin&utm_source=weixin&utm_medium=toutiao_android&utm_campaign=client_share&share_token=96d5f10d-7b63-43cd-be1d-b4c0152182ba&source=m_redirect)Baichuan2-192K的文本生成质量、长上下文理解以及长文本问答、摘要等方面的表现也全面领先Claude2。
- 模型在modelscope上的[主页](https://modelscope.cn/models/baichuan-inc/Baichuan2-13B-Chat/summary)。

In [1]:
## 官方示例
import torch
from modelscope import snapshot_download, AutoModelForCausalLM, AutoTokenizer,GenerationConfig
model_dir = "../huggingface/Baichuan2-13B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_dir, device_map="auto", 
                              trust_remote_code=True, torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", 
                              trust_remote_code=True, torch_dtype=torch.float16)
model.generation_config = GenerationConfig.from_pretrained(model_dir)

2023-10-30 15:41:15,190 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-10-30 15:41:15,193 - modelscope - INFO - Loading ast index from /home/user/.cache/modelscope/ast_indexer
2023-10-30 15:41:15,305 - modelscope - INFO - Loading done! Current index file version is 1.9.4, with md5 450c3d8b1ea957822a87246807bcd95c and a total number of 945 components indexed


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:

messages = []
messages.append({"role": "user", "content": "讲解一下“温故而知新”"})
response = model.chat(tokenizer, messages)
print(response)
messages.append({'role': 'assistant', 'content': response})
messages.append({"role": "user", "content": "背诵一下将进酒"})
response = model.chat(tokenizer, messages)
print(response)

"温故而知新"是一句源自《论语·为政》的古语，这句话的意思是：通过回顾过去，可以更好地理解现在和预见未来。它强调了学习和思考的重要性，鼓励我们在不断地回顾和总结过去的经验的基础上，去发现新的知识和见解。

这句话可以分为两部分来理解：

1. 温故：这里的“故”指的是过去的事情、知识或者经验。温故意味着我们要经常回顾和梳理过去的经历，以便更好地理解和掌握它们。

2. 知新：这里的“新”指的是新的知识、见解或者启示。知新意味着在温故的过程中，我们可以从中获得新的领悟和启发，从而不断提高自己的认知水平和能力。

总的来说，"温故而知新"是一种学习态度和方法，它鼓励我们不断地回顾和总结过去的经验，以便从中汲取新的知识和智慧，从而实现自我成长和提升。
君不见黄河之水天上来，奔流到海不复回。
君不见高堂明镜悲白发，朝如青丝暮成雪。
人生得意须尽欢，莫使金樽空对月。
天生我材必有用，千金散尽还复来。
烹羊宰牛且为乐，会须一饮三百杯。
岑夫子，丹丘生，将进酒，杯莫停。
与君歌一曲，请君为我倾耳听。
钟鼓馔玉不足贵，但愿长醉不复醒。
古来圣贤皆寂寞，惟有饮者留其名。
陈王昔时宴平乐，斗酒十千恣欢谑。
主人何为言少钱，径须沽取对君酌。
五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。


In [3]:
## 测试示例
messages = []
messages.append({"role": "user", "content": "鲁迅和周树人是什么关系？"})
response = model.chat(tokenizer, messages)
print(response)
messages.append({'role': 'assistant', 'content': response})
messages.append({"role": "user", "content": "张三的妈妈是李四的姐姐，张三和李四之间是什么关系？"})
response = model.chat(tokenizer, messages)
print(response)# 也是在胡说八道

鲁迅（1881年9月25日－1936年10月19日），原名周树人，字豫才。他是浙江绍兴人，是中国现代文学的奠基人和代表人物之一。他是一位杰出的作家、散文家、翻译家、文学评论家、编辑、医生等多重身份的人。他的作品如《狂人日记》、《呐喊》等对中国现代文学产生了深远的影响。因此，鲁迅是周树人的笔名，他们实际上是同一个人。
张三和李四之间是堂兄弟姐妹关系。因为张三的妈妈是李四的姐姐，所以他们有共同的祖先（他们的奶奶或外祖母），这是堂兄弟姐妹关系的定义。


### 16.2.4 Open LLM Leaderboard排行榜
- 截止本视频制作日期2023年10月20日，huggingface的Open LLM Leaderboard上排名第一的模型为Shining Valiant 1.2，查看[完整排行榜](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)。

- 部分排名如下：
<img src='../img/16_2_18.png' width=800px>


### 16.2.5 中文大语言模型
- 列表数据来自[LLMs-In-China](https://github.com/wgwang/LLMs-In-China)。


|序号|公司|大模型|省市|类别|官网|说明|
|:-|:-|:-|:-|:-|:-|:-|
|1|百度| [文心一言](https://yiyan.baidu.com)|北京|通用|✔|有APP，衍生[灵医Bot](https://01.baidu.com/)|
|2|智谱华章|[清言](https://chatglm.cn/)|北京|通用|✔|有APP，开源小模型[ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B)和[ChatGLM2-6B](https://github.com/THUDM/ChatGLM2-6B)|
|3|百川智能|[百川](https://chat.baichuan-ai.com/)|北京|通用|✔|开源小模型[baichuan-7B](https://github.com/baichuan-inc/baichuan-7B)和[Baichuan-13B](https://github.com/baichuan-inc/Baichuan-13B)，[baichuan-2](https://github.com/baichuan-inc/Baichuan2)|
|4|达观数据|[曹植](http://www.datagrand.com/products/aigc/)|上海|工业|✔|[试用需账号](https://aigc.datagrand.com/) |
|5|上海人工智能实验室|[书生](https://intern-ai.org.cn/) |上海|通用|✔|开源小模型[书生·浦语](https://github.com/InternLM/InternLM)，[OpenMEDLab浦医](https://github.com/openmedlab)|
|6|科大讯飞|[星火](https://xinghuo.xfyun.cn)|安徽合肥|通用|✔|[试用需账号](https://xinghuo.xfyun.cn/desk),有APP |
|7|稀宇科技|[ABAB](https://api.minimax.chat/)|上海|通用|✔|GLOW虚拟社交,MiniMax|
|8|商汤科技| [日日新](https://techday.sensetime.com/)| 上海|通用|✔||
|9|春田知韵（抖音）|[豆包](https://www.doubao.com/chat/)|北京|通用|✔|开源多模态7B小模型[BuboGPT](https://bubo-gpt.github.io/)，豆包是云雀的聊天机器人|
|10|中国科学院自动化研究所| [紫东·太初](https://gitee.com/zidongtaichu/multi-modal-models)|北京|通用|✔|紫东太初2.0号称100B参数，全模态|
|11|阿里云|[通义千问](https://tongyi.aliyun.com/)|浙江杭州|通用|✔|[试用需账号]( https://tongyi.aliyun.com),开源小模型[Qwen-7B](https://github.com/QwenLM/Qwen-7B)和[Qwen-7B-Chat](https://huggingface.co/Qwen/Qwen-7B-Chat)|
|12|华为|[盘古](https://openi.pcl.ac.cn/PCL-Platform.Intelligence/PanGu-Alpha),[盘古气象](https://www.nature.com/articles/s41586-023-06185-3),[盘古-Σ](https://arxiv.org/pdf/2303.10845.pdf)|广东深圳|工业|✔|华为+鹏城,[华为云盘古](https://www.huaweicloud.com/product/pangu.html)|
|13|复旦大学|[MOSS](https://github.com/OpenLMLab/MOSS)|上海|科研|✔|[试用需账号](https://moss.fastnlp.top/)|
|14|智源人工智能研究院|[悟道·天鹰](https://github.com/FlagAI-Open/FlagAI/tree/master/examples/Aquila),[悟道·EMU](https://github.com/baaivision/Emu)|北京|通用|✔|悟道3.0,视界视觉，AQUILA天鹰座，[Aquila-7B](https://model.baai.ac.cn/model-detail/100098),[AquilaChat-7B](https://model.baai.ac.cn/model-detail/100101),[AquilaCode-7B-NV](https://model.baai.ac.cn/model-detail/100102),[AquilaCode-7B-TS](https://model.baai.ac.cn/model-detail/100099),[HuggingFace](https://huggingface.co/BAAI),[EMU](https://huggingface.co/BAAI/Emu)基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)|
|15|浙江大学|[启真](https://github.com/CMKRG/QiZhenGPT),[TableGPT](https://github.com/syw1996/TableGPT),[智海-录问](https://github.com/zhihaiLLM/wisdomInterrogatory),[智海-三乐](),[PromptProtein](https://github.com/HICAI-ZJU/PromptProtein)|浙江杭州|垂直|✔|医学大模型提供基于LLaMA-7B、CaMA-13B和ChatGLM-6B 三个版本,用于PromptProtein的[模型](https://github.com/HICAI-ZJU/OpenProtein)，法律大模型智海-录问基于[Baichuan-7B](https://mp.weixin.qq.com/s/3PrA60M2T_lyCI25UXhIxg)，智海-三乐基于Qwen-7B|
|16|OpenBMB|[CPM](https://live.openbmb.org/),[CPM-Bee](https://github.com/OpenBMB/CPM-Bee)|北京|通用|✔|[面壁智能](https://modelbest.cn/),[CPM-Bee-10B](https://huggingface.co/openbmb/cpm-bee-10b)|
|17|元象科技|[XVERSE-13B](https://github.com/xverse-ai/XVERSE-13B)|广东深圳|通用|✔|[模型下载](https://huggingface.co/xverse/XVERSE-13B)|
|18|腾讯|[混元](https://hunyuan.tencent.com/)|广东深圳 |通用|✔ | |
|19|云知声|[山海](https://shanhai.unisound.com/) | 北京|医学|✔||
|20|东北大学|[TechGPT](https://github.com/neukg/TechGPT),[PICA](https://github.com/NEU-DataMining/PICA)|辽宁沈阳|科研|✔|TechGPT->BELLE->[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)，图谱构建和阅读理解问答;PICA->ChatGLM2-6B情感大模型|
|21|IDEA研究院|[封神榜MindBot](https://fengshenbang-lm.com/),[ziya-coding](https://huggingface.co/IDEA-CCNL/Ziya-Coding-15B-v1) |广东深圳|通用|✔|[姜子牙](https://huggingface.co/IDEA-CCNL/Ziya-LLaMA-13B-v1.1)系列模型 ,ziya-coding代码大模型|
|22|贝壳|[BELLE](https://github.com/LianjiaTech/BELLE)|北京|垂直|✔|基于BLOOMZ或[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)的多个模型|
|23|360| [智脑](https://ai.360.cn/),[一见](https://github.com/360CVGroup/SEEChat)| 北京|通用|✔||
|24|哈尔滨工业大学|[本草](https://github.com/SCIR-HI/Huatuo-Llama-Med-Chinese),[活字](https://github.com/hit-scir/huozi)|黑龙江哈尔滨|医学|✔|医学，本草基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)；另有基于 ChatGLM 的[Med-ChatGLM](https://github.com/SCIR-HI/Med-ChatGLM)，活字基于BLOOM-7B|
|25|北京大学信息工程学院|[ChatLaw](https://github.com/PKU-YuanGroup/ChatLaw)|北京|法律|✔|[ChatLaw-13B](https://huggingface.co/JessyTsu1/ChatLaw-13B)基于Ziya-LLaMA-13B-v1->LLaMA,[ChatLaw-33B](https://huggingface.co/JessyTsu1/ChatLaw-33B)基于Anima33B->Guanaco->[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)|
|26|港中文深圳|[华佗](https://github.com/FreedomIntelligence/HuatuoGPT)，[凤凰](https://github.com/FreedomIntelligence/LLMZoo)|广东深圳|医学|✔|香港中文大学（深圳）和深圳市大数据研究院，医学,[Demo](https://www.huatuogpt.cn/),华佗和凤凰都基于BLOOMZ|
|27|中国科学院计算技术研究所|[百聆](https://github.com/ictnlp/BayLing)|北京|科研|✔|基于 [LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)，权重Diff下载[7B](https://huggingface.co/ICTNLP/bayling-7b-diff)和[13B](https://huggingface.co/ICTNLP/bayling-13b-diff),[demo](http://nlp.ict.ac.cn/bayling/demo/) |
|28|好未来|[MathGPT](https://www.mathgpt.com/) |北京|教育|✔|学而思|
|29|晓多科技+国家超算成都中心|[晓模型XPT](https://xpt.xiaoduoai.com)|四川成都|客服|✔|[试用申请](https://xpt.xiaoduoai.com/)|
|30|网易有道|[子曰](https://aicenter.youdao.com/)|北京|教育|✔|推荐[有道速读](https://read.youdao.com/),读论文的利器|
|31|中国科学院成都计算机应用研究所|[聚宝盆](https://github.com/jerry1993-tech/Cornucopia-LLaMA-Fin-Chinese)|四川成都|金融|✔|基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)的金融大模型|
|32|华南理工大学|[扁鹊](https://github.com/scutcyr/BianQue),[灵心SoulChat](https://github.com/scutcyr/SoulChat)|广东广州|医学|✔| |
|33|虎博科技|[TigerBot](https://github.com/TigerResearch/TigerBot)|上海|金融|✔|基于[BLOOM](https://mp.weixin.qq.com/s/ia-yrmXbnlooRA3K1hoTwQ)|
|34|度小满|[轩辕](https://huggingface.co/xyz-nlp/XuanYuan2.0) |北京|金融|✔|基于[BLOOM](https://mp.weixin.qq.com/s/ia-yrmXbnlooRA3K1hoTwQ)|
|35|北京交通大学|[致远](https://github.com/DUOMO/TransGPT)|北京|交通|✔|[TransGPT・致远](https://huggingface.co/DUOMO-Lab/TransGPT-v0)，基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)-7B|
|36|恒生电子|LightGPT|浙江杭州|金融|✘| [与浙大合作的NL2SQL](https://arxiv.org/abs/2307.07306)|
|37|上海交通大学|[K2](https://github.com/davendw49/k2),[白玉兰](https://mp.weixin.qq.com/s/3eON8L4b7-d-1URwgdR6Bg)|上海|K2:地球科学，白玉兰:科学|✔|[Demo](https://k2.acemap.info/)，GeoLLaMA，基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)，[HuggingFace](https://huggingface.co/daven3/k2_it_adapter) |
|38|左手医生|[左医GPT](https://mp.weixin.qq.com/s/Tv9nIG_9K-Lf5AKatjichA)|北京|医学|✔|医疗，[试用需Key](https://gpt.zuoshouyisheng.com/)|
|39|上海科技大学|[DoctorGLM](https://github.com/xionghonglin/DoctorGLM)|上海|医学|✔|医学大模型，[论文](https://arxiv.org/pdf/2304.01097.pdf)|
|40|华东师范大学 |[EmoGPT](https://mp.weixin.qq.com/s/xP-qm5YUj8fZD9YQ7t08NQ),[EduChat](https://github.com/icalk-nlp/EduChat)|上海|教育| ✘|EmoGPT是上海市心理健康与危机干预重点实验室与镜象科技公司合作完成, 教学教育大模型EduChat基于BELLE（BELLE基于LLaMA）|
|41|艾写科技|[Anima](https://github.com/lyogavin/Anima)|浙江杭州|营销|✔|基于Guanaco->基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA)，使用QLoRA|
|42|澳门理工大学|[XrayGLM](https://github.com/WangRongsheng/XrayGLM),[IvyGPT](https://github.com/WangRongsheng/IvyGPT)|澳门|医疗|✔|IvyGPT基于ChatGLM2，XrayGLM基于VisualGLM-6B|
|43|北京语言大学 |[桃李](https://github.com/blcuicall/taoli) |北京|教育|✔|基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA),北语+清华+东北、北京交大|
|44|中工互联|[智工](https://mp.weixin.qq.com/s/ANsZeqj4V_NeVCquwX-aSQ)|北京|工业|✘|与复旦NLP实验室联合，工业领域|
|45|创业黑马|[天启](https://mp.weixin.qq.com/s/lYqCe9skc0MOSzmmTiGAug)|北京 |创投|✘ | 创业黑马与360合作,科创服务行业|
|46|追一科技|[博文Bowen](https://mp.weixin.qq.com/s/cYVh6K6edmColgMEOaGFKg)|广东深圳|客服 |✘ | |
|47|智慧眼|[砭石](https://mp.weixin.qq.com/s/lid0nUBwXEdoUhnw_guteA)|湖南长沙|医学|✘|医疗领域|
|48|香港科技大学|[罗宾Robin](https://huggingface.co/OptimalScale)|香港|科研|✔|基于[LLaMA](https://mp.weixin.qq.com/s/dKInMi6P80GXecUtR3WQsA),[港科大开源LMFlow](https://github.com/OptimalScale/LMFlow)|
|49|昆仑万维 | [天工](https://github.com/SkyWorkAIGC)| 北京|客服|✔|与奇点智源联合研发||
|50|智媒开源研究院| [智媒](https://github.com/IMOSR/Media-LLaMA)|广东深圳|媒体|✔|基于LLaMA，面向自媒体|
|51|医疗算网|Uni-talk|上海|医学|✘|上海联通+华山医院+上海超算中心+华为|
|52|蚂蚁集团|贞仪,[CodeFuse](https://huggingface.co/codefuse-ai)|浙江杭州|金融|✔|CodeFuse代码大模型|
|53|硅基智能|[炎帝](https://mp.weixin.qq.com/s/XNu3UrSKm4jy1ayJJ6-HMg)|江苏南京|文旅|✘||
|54|西湖心辰|[西湖](https://xinchenai.com/)|浙江杭州|科研|✔| |
|55|国家超级计算天津中心|[天河天元](https://mp.weixin.qq.com/s/A9jnnL3-LjcDLsDD2PCa6g)|天津|通用|✘ | |
|56|星环科技|[无涯、求索](https://mp.weixin.qq.com/s/6rYmk58OypU_Wwu0L7-nTw)|上海|金融|✘ |无涯——金融；求索——大数据分析|
|57|清博智能 | [先问](https://mp.weixin.qq.com/s/Et-nVHjxDP3W-PFWmDo3YQ)| 北京|农业|✘|基于结构化数据|
|58|智子引擎 | [元乘象](https://chatimg.aixiaoqingxu.com/)|江苏南京 |客服|✔||
|59|拓世科技 |[拓世](https://tskjgroup.com/article_news?id=822) |江西南昌 |金融|✘||
|60|循环智能| [盘古](https://www.rcrai.com/product/ai/pangu)|北京|客服|✔|循环智能,清华大学,华为 |
|61|慧言科技+天津大学|[海河·谛听](https://mp.weixin.qq.com/s/FCnXXmT0jRfk4tTRIAK9FA) |天津|科研 |✘| |
|62|第四范式|[式说](https://www.4paradigm.com/product/SageGPT.html)|北京|客服|✔||
|63|拓尔思|[拓天](https://mp.weixin.qq.com/s/beQardxjpner6vvk_LTOJA)|北京 |媒体|✘ |TRSGPT |
|64|出门问问|[序列猴子](https://write.mobvoi.com/) |北京 |营销|✔| |
|65|数说故事|[SocialGPT](https://mp.weixin.qq.com/s/Tt3dcwefIvdlyB_IwaFYRQ)|广东广州|社交|✘| |
|66|云从科技|[从容](https://www.cloudwalk.com/news/show/id/178)|广东广州|政务|✔||
|67|浪潮信息|[源](https://air.inspur.com/) |山东济南|通用|✘|[源](https://github.com/Shawn-Inspur/Yuan-1.0)|
|68|中国农业银行|[小数ChatABC](https://mp.weixin.qq.com/s/CXyZRIqhwrcGAKxzUC-qgg) |北京|金融|✘| |
|69|麒麟合盛 |[天燕AiLMe](https://www.apusai.com/) |北京|运维|✔| |
|70|台智云|[福尔摩斯FFM](https://tws.twcc.ai/afs/) |台湾|工业|✔|华硕子公司|
|71|医联科技|[medGPT](https://www.medlinker.com/news/198)|四川成都|医学|✘| |
|72|电信智科|[星河](https://mp.weixin.qq.com/s/ntd0z5CJOY6peou4bOVJqA)|北京|通信|✘|通用视觉，中国电信|
|73|深思考人工智能|[Dongni](https://www.dongni.ai/)|北京|媒体|✔| |
|74|文因互联|文因|安徽合肥|金融|✘| 金融大模型|
|75|印象笔记|[大象GPT](https://mp.weixin.qq.com/s/O3nRSKBM29bCWKuRRi_PBg)|北京 |媒体| ✘| |
|76|中科闻歌|[雅意](https://mp.weixin.qq.com/s/IGYV3t3JRlq4quvNJmZ4vA)|北京|媒体|✘||
|77|澜舟科技| [孟子](https://www.langboat.com/portal/mengzi-model) | 北京|金融|✔||
|78|京东|[言犀](https://www.jdcloud.com/cn/news/detail/1235)|北京|商业|✘||
|79|香港中文大学|[PointLLM](https://github.com/OpenRobotLab/PointLLM)|香港|通用|✔|港中文+上海AI实验室+浙大|
|80|清华大学|[NowcastNet](https://www.nature.com/articles/s41586-023-06184-4)|北京|科研|✔| [气象,临近预报大模型](https://mp.weixin.qq.com/s/MwJWjCLNqJM3lZ33RwK4Bg)|
|81|鹏城实验室|鹏城·脑海|广东深圳|科研|✘|Peng Cheng Mind|
|82|宇视科技|[梧桐](https://mp.weixin.qq.com/s/H8FsrEyJsIijy0Cowyu3GQ)|浙江杭州|运维|✘|AIoT行业 |
|83|智臻智能|[华藏](https://mp.weixin.qq.com/s/MZO2tvun05WnJkSe0seJnw)|上海|客服|✘|小i机器人|
|84|美亚柏科|[天擎](https://mp.weixin.qq.com/s/D3ki3G4Q7QZPAVJ8iwTvDg)|福建厦门 |安全|✘|公共安全|
|85|山东大学|[夫子•明察](https://github.com/irlab-sdu/fuzi.mingcha)|山东济南|司法|✔|山东大学+浪潮云+中国政法大学，基于ChatGLM，无监督司法语料（各类判决文书、法律法规等）与有监督司法微调数据（包括法律问答、类案检索）训练而成|
|86|数慧时空|[长城](https://mp.weixin.qq.com/s/KYB-noOt7gB0l5hh-rwfkQ)|北京|地球科学 |✘ |自然资源，遥感|
|87|佳都科技 |佳都知行 | 广东广州|交通|✘|交通领域|
|88|知乎|知海图|北京|媒体|✘ |知乎和面壁科技合作|
|89|网易伏羲|玉言|广东广州 |通用|✘ | |
|90|清睿智能|[ArynGPT](https://mp.weixin.qq.com/s/FFRfzwoXBM2dGs9O7F-Z5A) | 江苏苏州|教育|✘||
|91|微盟|[WAI](https://wai.weimob.com/)|上海|商业|✔||
|92|西北工业大学+华为 |[秦岭·翱翔](https://www.nwpu.edu.cn/info/1198/65828.htm) |陕西西安|工业|✘|流体力学大模型,湍流+流场 |
|93|奇点智源| [天工智力](https://openapi.singularity-ai.com/)|北京 |通用|✔|[瑶光和天枢](https://openapi.singularity-ai.com/index.html#/documentIndex)|
|94|联汇科技 |欧姆|浙江杭州 |通用|✔|[OmModel欧姆多模态（视觉语言）大模型](https://om.linker.cc/)|
|95|中国联通 |鸿湖|北京|通信|✘||
|96|思必驰|[DFM-2](https://mp.weixin.qq.com/s/FxLw5UfJpYS1tCPDMkhvXA)|江苏苏州|工业|✘||
|97|理想科技|大道Dao|北京|运维|✘|运维大模型|
|98|电科太极 |[小可](https://mp.weixin.qq.com/s/8ci7g7R9j3pxkQC4UOLh2A) |北京|政务|✘|党政企行业应用 |
|99|中国移动|九天| 北京|通信|✘||
|100|中国电信|TeleChat|北京|通信|✘||
|101|容联云|赤兔|北京|客服|✘|客服，营销|
|102|云天励飞|天书|广东深圳|政务|✘||
|103|乐言科技|乐言 |上海 |客服| ✘| |
|104|沪渝人工智能研究院|[兆言](https://mp.weixin.qq.com/s/RtcdWGrfIW5unyvxVplCSg)|重庆|科研|✘|也称：上海交通大学重庆人工智能研究院|
|105|中央广播电视总台|央视听|北京|媒体|✘|央视听媒体大模型CMG Media GPT|
|106|超对称技术公司| [乾元](https://bbt.ssymmetry.com/)|北京|金融|✔||
|107|蜜度|[文修](https://mp.weixin.qq.com/s/aHSw9Kxib3Zj84qDGn3Dyg)|上海|媒体|✘|智能校对|
|108|中国电子云|[星智](https://mp.weixin.qq.com/s/qNoTD4BY2DX5ziSe1ZPSLw)|湖北武汉|政务|✘|政务大模型|
|109|理想汽车|MindGPT|北京 |工业|✘| |
|110|阅文集团|妙笔|上海|文旅|✘| 网文大模型|
|111|携程|问道|上海|文旅|✘|旅游行业大模型|
|112|实在智能 |塔斯|浙江杭州|客服|✘| TARS|
|113|瑞泊|[VIDYA](http://xraybot.com/col.jsp?id=103)|北京|工业|✔| |
|114|有连云|麒麟|上海|金融|✘| |
|115|维智科技|CityGPT|上海|公共服务|✘|城市大模型|
|116|用友|YonGPT|北京|企业服务|✘| |
|117|天云数据|Elpis|北京|金融|✘|证券法律法规|
|118|孩子王|KidsGPT|江苏南京|教育|✘| |
|119|企查查|知彼阿尔法 |江苏苏州 |商业|✘||
|120|今立方|12333|福建厦门|政务|✘|人社领域|
|121|阳光保险集团|正言|广东深圳|金融|✘||
|122|中科创达|魔方Rubik|北京|工业|✘| |
|123|聆心智能|CharacterGLM|北京|游戏|✘||
|124|大经中医|岐黄问道|江苏南京|医疗|✘| |
|125|蒙牛|MENGNIU.GPT|内蒙古呼和浩特|食品|✘| |
|126|快商通|汉朝|福建厦门|营销|✘| |
|127|众合科技|UniChat|浙江杭州|交通|✘| |
|128|金蝶|苍穹|广东深圳|企业服务|✘| |
|129|云问科技|云中问道|江苏南京|营销|✘|与西安未来AI计算中心联合发布|
|130|天壤智能|小白|上海|通用|✘| |
|131|小米|MiLM-6B|北京|商业|✘| |
|132|长虹|长虹超脑 |四川绵阳|媒体|✘| |
|133|开普云|[开悟](https://www.kaipuyun.cn/kp/c100630/kaiwu.shtml)|广东东莞|政务|✔| |
|134|赛灵力科技 |达尔文|广东广州|医学|✘|赛灵力,清华珠三角研究院,赛业生物,大湾区科技创新服务中心|
|135|航旅纵横|千穰大模型|北京|民航|✘|航旅纵横APP上需要PLUS会员才能使用|
|136|奇安信|Q-GPT|北京|信息安全|✘| |
|137|车之谷|叆谷|山东青岛|汽车|✘|汽车后服务加油站场景|
|138|索贝时代|明眸|四川成都|媒体|✘| |
|139|海尔|HomeGPT|山东青岛|智能家居|✘| |
|140|马上消费|天镜|重庆|金融|✘|零售金融|
|141|白海科技|白聚易|北京|营销|✘|营销传播专家多模态预训练模型IMC-GPT（白聚易）|
|142|二元工业|妆舟|江苏苏州|日化|✘|回答化妆、护肤和服饰搭配等问题，日化行业从业人员提供从产品开发、行业服务到品牌建设等指导|
|143|格创东智|章鱼智脑|广东广州|工业制造|✘|工业智能大模型引擎底座——章鱼智脑OctopusGPT|
|144|创业邦|BangChat|北京|创投|✘|产业、企业和投资行业|
|145|新华三H3C|百业灵犀|浙江杭州|工业|✘||
|146|作业帮|银河|广东广州|教育|✘||
|147|电科数字|智弈|上海|水利|✘||
|148|绿盟|风云卫|北京|网络安全|✘|NSFGPT|
|149|江苏欧软|WISE|江苏苏州|工业|✘|WISE工业大模型|
|150|创新奇智|奇智孔明|山东青岛|工业|✘|工业大模型AInno-15B，ChatRobot，ChatBI，ChatDoc|
|151|大汉软件|星汉|江苏南京|政务|✘|“星汉”Galaxy大模型|
|152|零点有数|零点楷模|北京|政务|✘||
|153|国农生猪大数据中心|PIGGPT|重庆|农业|✘||
|154|微脉|CareGPT|浙江杭州|医疗|✘||
|155|吉大正元|昆仑|吉林长春|信息安全|✘||
|156|武汉大学|[CheeseChat](https://www.ithome.com/0/695/295.htm)|湖北武汉|教育|✘|内测招募，仅限武汉大学在校师生申请|
|157|方正电子|魔方|北京|媒体|✘|聚焦媒体市场需求|
|158|似然实验室|TraderGPT|广东广州|金融|✘|金融持仓分析大模型|
|159|网易智企|商河|广东广州|客服|✘|客服领域行业大模型|
|160|深圳供电局|祝融2.0|广东深圳|电力|✘|电力行业首个多模态预训练大模型|
|161|万兴科技|天幕|西藏拉萨|媒体|✘|以视频创意应用为核心|
|162|惟远智能|千机百智|广东深圳|客服|✘||
|163|兔展智能|兔灵|广东深圳|营销|✘||
|164|中国科学技术大学|[UniDoc](https://arxiv.org/pdf/2308.11592.pdf)|安徽合肥|通用|✘|中科大&字节,统一的文字-图像理解大模型|
|165|钢谷网|谷蚁|陕西西安|电商|✘|钢铁行业电商|
|166|浪潮海岳|inGPT|山东济南|企业服务|​✘||
|167|木卫四科技|蝴蝶|北京|汽车|✘| |
|168|汇通达网络|汇通达|江苏南京|企业服务|✘|下沉市场零售行业企业客户的交易和服务的互联网平台,农村电商服务|

- [Awesome-Chinese-LLM](https://github.com/HqWu-HITCS/Awesome-Chinese-LLM)对中文大语言模型及其周边也进行相当不错的归纳和总结。